In [ ]:
import ee 

ee.Initialize()

In [ ]:
l1 = ee.List(["a", "b", "c"]).map(lambda e: ee.String(e))
l2 = ee.List([1,  2, 3]).map(lambda e: ee.String(e))

l1.map(lambda e: l2.map(lambda f: ee.String(e).cat(ee.String(f)))).flatten().getInfo()